In [1]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [2]:
mrf = gum.MarkovRandomField()

## Add variables to the MRF

Concept 0: percentage of students

Concept 1: number of students

Concept 2: total number

Concept 3: percentage of student population

In [3]:
NUM_CONCEPTS = 4
variables = []

for i in range(1, NUM_CONCEPTS):
    for j in range(i+1, NUM_CONCEPTS+1):
        var = gum.IntegerVariable(f"r{i}{j}", f"relation variable between concept {i} and {j}", [0, 1])

        mrf.add(var)
        variables.append(var)

print("MRF variables: ", mrf.names())
print("MRF factors: ", mrf.factors())

MRF variables:  {'r12', 'r14', 'r24', 'r23', 'r13', 'r34'}
MRF factors:  []


## Add unary factors

In [4]:
# real unary factors
table_unary = [[0.05, 0.95], # 0.05, 0.95
               [0.97, 0.03],
               [0.3, 0.7], # 0.30, 0.70
               [0.14, 0.86],
               [0.87, 0.13],
               [0.96, 0.04]]

for i, var in enumerate(variables):
    p = gum.Potential().add(var).fillWith(table_unary[i])
    mrf.addFactor(p)

mrf.factors()

[{1}, {3}, {5}, {0}, {2}, {4}]

## Add binary factors

In [5]:
table_binary = [0.5, 0.5, 0.5, 0.5]
binary_factors = [(0, 1), (0, 2), (0, 3), (0, 4),
                  (1, 2), (1, 3), (1, 5),
                  (2, 4), (2, 5),
                  (3, 4), (3, 5),
                  (4, 5)]

for (i, j) in binary_factors:
    p = gum.Potential().add(variables[i]).add(variables[j]).fillWith(table_binary)
    mrf.addFactor(p)

mrf.factors()

[{1, 3},
 {1, 5},
 {3, 5},
 {1, 2},
 {0, 3},
 {0, 1},
 {5},
 {3},
 {1},
 {2, 5},
 {3, 4},
 {4, 5},
 {0, 4},
 {0, 2},
 {4},
 {2},
 {0},
 {2, 4}]

## Add ternary factors

In [6]:
table_ternary = [1, 1, 1, 0, 1, 0, 0, 1]
ternary_factors = [(0, 1, 3), (0, 2, 4),
                   (1, 2, 5),
                   (3, 4, 5)]

for (i, j, k) in ternary_factors:
    p = gum.Potential().add(variables[i]).add(variables[j]).add(variables[k]).fillWith(table_ternary)
    mrf.addFactor(p)

mrf.factors()

[{1, 3},
 {1, 5},
 {3, 5},
 {0, 1, 3},
 {1, 2, 5},
 {3, 4, 5},
 {1, 2},
 {0, 3},
 {0, 1},
 {5},
 {3},
 {1},
 {2, 5},
 {3, 4},
 {4, 5},
 {0, 4},
 {0, 2},
 {4},
 {2},
 {0},
 {2, 4},
 {0, 2, 4}]

## Markov Random Field and factor graph

In [7]:
gnb.sideBySide(gnb.getMRF(mrf, view="graph", size="5"),
               gnb.getMRF(mrf, view="factorgraph", size="10"))

## Shafer-Shenoy Inference

In [8]:
ss = gum.ShaferShenoyMRFInference(mrf)
ss.makeInference()

In [9]:
gnb.sideBySide(*[ss.posterior(f"r{i}{j}") for i in range(1, NUM_CONCEPTS) for j in range(i+1, NUM_CONCEPTS+1)])

In [10]:
# mrf = gum.fastMRF("r12;r13;r14;r23;r24;r34;r12--r13;r12--r14;r12--r23;r12--r24;r13--r14;r13--r23;r13--r34;r14--r24;r14--r34;r23--r24;r23--r34;r24--r34;r12--r13--r23;r12--r14--r24;r13--r14--r34;r23--r24--r34")

In [11]:
# gnb.sideBySide(gnb.getMRF(mrf,view="graph",size="5"),
#                gnb.getMRF(mrf,view="factorgraph",size="5"))

In [12]:
# gnb.showMRF(mrf, size="10")

In [13]:
# iemn = gum.ShaferShenoyMRFInference(mrf)
# iemn.setEvidence({'r12':1,'r23':1,'r34':1})